In [1]:
# import packages and change directory
import pandas as pd

import os

os.chdir('/home/workgroups/socialclass')

os.getcwd()



'/home/workgroups/socialclass'

In [2]:
true_pairs = pd.read_stata("./data/A1_1_couples_info.dta")
men_df = pd.read_stata("./data/A1_1_couples_info_man.dta")
women_df = pd.read_stata("./data/A1_1_couples_info_woman.dta")

true_pairs = true_pairs[true_pairs['t_match'] == 2007]
men_df = men_df[men_df['t_match'] == 2007]
women_df = women_df[women_df['t_match'] == 2007]


In [3]:
true_pairs.head()

,id_spouse,parent1,parent2,parent11,parent12,parent21,parent22,lob1,lob2,lob11,...,wealth12,wealth21,wealth22,educ1,educ2,educ11,educ12,educ21,educ22,t_match
16,63.0,3708472.0,6219584.0,11597345.0,1905940.0,3965608.0,16182534.0,1.0,18.0,21.0,...,-79619.0,319090.0,17107.0,3.0,6.0,4.0,6.0,6.0,5.0,2007
64,248.0,3576416.0,13369917.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,2007
113,448.0,107878.0,14863115.0,2023085.0,10344587.0,14942510.0,13167454.0,1.0,1.0,1.0,...,149830.0,-11682.0,-45421.0,3.0,5.0,3.0,2.0,1.0,2.0,2007
170,718.0,8935345.0,15305928.0,4241820.0,629776.0,13688758.0,10671020.0,11.0,11.0,11.0,...,35731.0,465391.0,460659.0,4.0,6.0,3.0,3.0,2.0,3.0,2007
184,772.0,5946164.0,15453082.0,13916143.0,10723143.0,4360691.0,4339676.0,12.0,13.0,12.0,...,-37470.0,4010198.0,1370298.0,3.0,6.0,1.0,1.0,3.0,6.0,2007


In [4]:
men_df.head()

,id_spouse,parent1,parent11,parent12,lob1,lob11,lob12,cob1,cob11,cob12,...,kommun1,kommun11,kommun12,wealth1,wealth11,wealth12,educ1,educ11,educ12,t_match
16,63.0,3708472.0,11597345.0,1905940.0,1.0,21.0,4.0,Sweden,Sweden,Sweden,...,0180,0114,0114,1170240.0,1282233.0,-79619.0,3.0,4.0,6.0,2007
64,248.0,3576416.0,NaN,NaN,NaN,NaN,NaN,Syrian Arab Republic,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2007
113,448.0,107878.0,2023085.0,10344587.0,1.0,1.0,2.0,Sweden,Sweden,Sweden,...,0188,0117,0117,-58032.0,138587.0,149830.0,3.0,3.0,2.0,2007
170,718.0,8935345.0,4241820.0,629776.0,11.0,11.0,11.0,Sweden,Sweden,Sweden,...,1293,1293,1293,2964.0,23219.0,35731.0,4.0,3.0,3.0,2007
184,772.0,5946164.0,13916143.0,10723143.0,12.0,12.0,13.0,Sweden,Sweden,Sweden,...,1283,NaN,1283,-107376.0,NaN,-37470.0,3.0,1.0,1.0,2007


In [5]:
women_df.head()

,id_spouse,parent2,parent21,parent22,lob2,lob21,lob22,cob2,cob21,cob22,...,kommun2,kommun21,kommun22,wealth2,wealth21,wealth22,educ2,educ21,educ22,t_match
16,63.0,6219584.0,3965608.0,16182534.0,18.0,8.0,NaN,Sweden,Sweden,Canada,...,0180,0880,0880,-168468.0,319090.0,17107.0,6.0,6.0,5.0,2007
64,248.0,13369917.0,NaN,NaN,NaN,NaN,NaN,Syrian Arab Republic,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2007
113,448.0,14863115.0,14942510.0,13167454.0,1.0,2.0,1.0,Sweden,Sweden,Sweden,...,0188,0117,0117,-25332.0,-11682.0,-45421.0,5.0,1.0,2.0,2007
170,718.0,15305928.0,13688758.0,10671020.0,11.0,11.0,11.0,Sweden,Sweden,Sweden,...,1293,1290,1290,0.0,465391.0,460659.0,6.0,2.0,3.0,2007
184,772.0,15453082.0,4360691.0,4339676.0,13.0,13.0,13.0,Sweden,Sweden,Sweden,...,1283,1380,1380,-298575.0,4010198.0,1370298.0,6.0,3.0,6.0,2007


In [13]:
import pandas as pd
import numpy as np

# PARAMETERS
K = 100
rng = np.random.default_rng(42)

# --- Step 1. Clean duplicates and merge to create positive pairs ---
men_df_clean = men_df.drop(columns=['t_match'], errors='ignore')
women_df_clean = women_df.drop(columns=['t_match'], errors='ignore')

true_pairs_full = (
    true_pairs
    .merge(men_df_clean, on='id_spouse', suffixes=('', '_m'))
    .merge(women_df_clean, on='id_spouse', suffixes=('_m', '_w'))
)
true_pairs_full['match'] = 1

# --- Step 2. Add partner province columns to men_df and women_df ---
# (so each man knows his wife's province and vice versa)
men_with_spouseprov = men_df.merge(
    true_pairs[['id_spouse', 'lan2']], on='id_spouse', how='left'
).rename(columns={'lan2': 'spouse_province'})

women_with_spouseprov = women_df.merge(
    true_pairs[['id_spouse', 'lan1']], on='id_spouse', how='left'
).rename(columns={'lan1': 'spouse_province'})

# --- Step 3. Efficient vectorized sampling ---
def sample_negatives(df_source, df_target, province_col, target_province_col, K):
    """Sample K random opposite-sex individuals from same province."""
    samples = []
    for prov, group_source in df_source.groupby(province_col):
        pool = df_target[df_target[target_province_col] == prov]
        if pool.empty:
            continue
        idx = rng.choice(pool.index, size=K * len(group_source), replace=True)
        sampled = pool.loc[idx].reset_index(drop=True)
        repeated = group_source.loc[group_source.index.repeat(K)].reset_index(drop=True)
        pairs = pd.concat([repeated.add_suffix('_m'), sampled.add_suffix('_w')], axis=1)
        pairs['match'] = 0
        samples.append(pairs)
    return pd.concat(samples, ignore_index=True) if samples else pd.DataFrame()

# --- Step 4. Generate negatives ---
# men-based negatives: women from same province as man's actual wife
neg_from_men = sample_negatives(
    df_source=men_with_spouseprov,
    df_target=women_df,
    province_col='spouse_province',
    target_province_col='lan2',
    K=K
)

# women-based negatives: men from same province as woman's actual husband
neg_from_women = sample_negatives(
    df_source=women_with_spouseprov,
    df_target=men_df,
    province_col='spouse_province',
    target_province_col='lan1',
    K=K
)

# Fix column name symmetry
neg_from_women = neg_from_women.rename(columns=lambda c: c.replace('_w', '_tmp') if c.endswith('_w') else c)
neg_from_women = neg_from_women.rename(columns=lambda c: c.replace('_m', '_w') if c.endswith('_m') else c)
neg_from_women = neg_from_women.rename(columns=lambda c: c.replace('_tmp', '_m') if c.endswith('_tmp') else c)

# --- Step 5. Combine all ---
all_pairs = pd.concat([true_pairs_full, neg_from_men, neg_from_women], ignore_index=True)

print("Final data shape:", all_pairs.shape)
print(all_pairs['match'].value_counts())

# Optional: save
# all_pairs.to_parquet("marriage_pairs_for_xgb.parquet")


/tmp/ipykernel_838449/1163345005.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for prov, group_source in df_source.groupby(province_col):
/tmp/ipykernel_838449/1163345005.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for prov, group_source in df_source.groupby(province_col):


Final data shape: (10965419, 130)
match
0    10904200
1       61219
Name: count, dtype: int64


In [14]:
print("Men with missing spouse_province:", men_with_spouseprov['spouse_province'].isna().sum())
print("Women with missing spouse_province:", women_with_spouseprov['spouse_province'].isna().sum())

# check how many provinces skipped due to no opposite-sex pool
missing_provinces = set(men_with_spouseprov['spouse_province']) - set(women_df['lan2'])
print("Provinces with men but no women:", missing_provinces)



Men with missing spouse_province: 6612
Women with missing spouse_province: 6784
Provinces with men but no women: set()


# Machine Learning Prediction Model: XGBoost

In [19]:
# === 0) Imports
import pandas as pd
import numpy as np

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

# If you haven’t already:
# pip install xgboost scikit-learn


In [20]:
# === 1) Basic setup

df = all_pairs.copy()

# Label
y = df['match'].astype(int)

# Identify ID columns to avoid leakage (keep them only for grouping, drop from features)
id_cols = [c for c in df.columns if c.lower().startswith('id_spouse')]
# In your screenshots, these exist: 'id_spouse', 'id_spouse_m', 'id_spouse_w'
# If some are missing, this will just ignore them.

# Build a group key so each person (either side) is not split across train/test
# Combine the two sides; fill NAs robustly.
man_id  = df[id_cols[0]] if 'id_spouse_m' in df else df.get('id_spouse', pd.Series(index=df.index, dtype='float'))
woman_id = df[id_cols[1]] if 'id_spouse_w' in df else df.get('id_spouse', pd.Series(index=df.index, dtype='float'))

# Best effort: if both 'id_spouse_m' and 'id_spouse_w' exist, use them; otherwise fall back to 'id_spouse'
man_id   = df.get('id_spouse_m', df.get('id_spouse'))
woman_id = df.get('id_spouse_w', df.get('id_spouse'))

groups = (man_id.astype('Int64').astype(str) + '-' + woman_id.astype('Int64').astype(str)).fillna('NA-NA')

# === 2) Choose feature columns
drop_cols = set(['match']) | set(id_cols)  # drop label and ID columns from features
X = df.drop(columns=list(drop_cols), errors='ignore')


In [21]:
# === 3) Make categoricals work nicely

# XGBoost can handle pandas 'category' dtype directly (enable_categorical=True).
# We'll cast:
#   - object dtype columns
#   - code-like columns that are categorical despite numeric (e.g., lan*, kommun*, cob*)
cat_like_prefixes = ('lan1','lan2','cob1','cob2','educ1','educ2','kommun1','kommun2','lob1','lob2')  # adjust as needed for your schema

categorical_cols = (
    list(X.select_dtypes(include=['object']).columns) +
    [c for c in X.columns
     if c.startswith(cat_like_prefixes) and str(X[c].dtype) not in ('float64','float32','int64','int32')]
)

# Even if numerically typed, force to category for the prefixes (these are province/municipality/etc. codes)
for c in X.columns:
    if c.startswith(cat_like_prefixes):
        X[c] = X[c].astype('category')

for c in categorical_cols:
    X[c] = X[c].astype('category')

# Make sure numeric columns are numeric
# (XGBoost will accept category + numeric together)


In [22]:
# === 4) Person-blocked train/test split to prevent leakage

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


In [23]:
# === 5) Handle class imbalance
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()
scale_pos_weight = neg / max(pos, 1)

print(f"Train positives: {pos:,}, negatives: {neg:,}, scale_pos_weight={scale_pos_weight:.2f}")


Train positives: 61,219, negatives: 8,723,346, scale_pos_weight=142.49


In [ ]:
# === 6) Train XGBoost
# Use 'hist' for speed; enable_categorical=True to use categorical splits.

clf = XGBClassifier(
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    objective='binary:logistic',
    eval_metric='auc',
    tree_method='hist',
    enable_categorical=True,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
)

clf.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=50,
    early_stopping_rounds=100
)


/home/ehsan/miniforge3/envs/marriage/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [22:12:10] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1758008603490/work/src/learner.cc:738: 
Parameters: { "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()


[0]	train-auc:1.00000	eval-auc:nan


/home/ehsan/miniforge3/envs/marriage/lib/python3.10/site-packages/xgboost/callback.py:264: UserWarning: [22:12:17] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1758008603490/work/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/home/ehsan/miniforge3/envs/marriage/lib/python3.10/site-packages/xgboost/callback.py:264: UserWarning: [22:12:18] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1758008603490/work/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/home/ehsan/miniforge3/envs/marriage/lib/python3.10/site-packages/xgboost/callback.py:264: UserWarning: [22:12:19] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1758008603490/work/src/metric/auc.cc:324: Dataset is empty, or contains only positive o

[50]	train-auc:1.00000	eval-auc:nan


/home/ehsan/miniforge3/envs/marriage/lib/python3.10/site-packages/xgboost/callback.py:264: UserWarning: [22:12:32] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1758008603490/work/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/home/ehsan/miniforge3/envs/marriage/lib/python3.10/site-packages/xgboost/callback.py:264: UserWarning: [22:12:33] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1758008603490/work/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  score: str = model.eval_set(evals, epoch, self.metric, self._output_margin)
/home/ehsan/miniforge3/envs/marriage/lib/python3.10/site-packages/xgboost/callback.py:264: UserWarning: [22:12:34] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1758008603490/work/src/metric/auc.cc:324: Dataset is empty, or contains only positive o

[99]	train-auc:1.00000	eval-auc:nan
Test ROC-AUC: nan


/home/ehsan/miniforge3/envs/marriage/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
